<a href="https://colab.research.google.com/github/shivanishimpi/blogs/blob/main/ANN_with_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> This notebook was created to demonstrate ANN with keras-tuner as an extension to the [keras-tuner blog](https://shivanishimpi9.medium.com/choosing-the-right-set-of-hyperparameters-keras-tuner-introduction-implementation-d4c387f4d655?postPublishedType=repub) 

To follow through the notebook explanation, join our YouTube [live session](https://www.youtube.com/watch?v=DQuS_cZg2wI)

Download the Churn_Modelling.csv file [here](https://drive.google.com/file/d/1ZC5IJ4DiRAFMoIX7t9JyU84IF8lUxa0q/view?usp=sharing)


Date Created: January 15, 2021



**Authors:**

1. Satyajit Pattnaik `ANN`
2. Shivani Shimpi `Keras-tuner`

Reach us out on GitHub / LinkedIn
- shivanishimpi9@gmail.com [[GitHub](https://github.com/shivanishimpi)] [[LinkedIn](https://www.linkedin.com/in/shivani-shimpi-5113a8170/)]
- pattnaiksatyajit89@gmail.com [[LinkedIn](https://www.linkedin.com/in/satyajit-pattnaik-b41392102/)]

In [ ]:
#@title Mount Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Blogs/Keras-tuner' #set as per your needs

/content/drive/MyDrive/Blogs/Keras-tuner


In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

print(X)

[[619 0 0 ... 1 1 101348.88]
 [608 2 0 ... 0 1 112542.58]
 [502 0 0 ... 1 0 113931.57]
 ...
 [709 0 0 ... 0 1 42085.58]
 [772 1 1 ... 1 0 92888.52]
 [792 0 0 ... 1 0 38190.78]]


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#Classical ML

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, pred)


print(cm)

[[1350  245]
 [ 185  220]]


In [ ]:
# Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third  hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
y_train[0]
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 25)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)



cm = confusion_matrix(y_test, y_pred)

print(cm)

Epoch 1/25
800/800 [==============================] - 2s 1ms/step - loss: 0.5861 - accuracy: 0.7968
Epoch 2/25
800/800 [==============================] - 1s 1ms/step - loss: 0.4282 - accuracy: 0.8033
Epoch 3/25
800/800 [==============================] - 1s 1ms/step - loss: 0.4121 - accuracy: 0.8208
Epoch 4/25
800/800 [==============================] - 1s 1ms/step - loss: 0.4063 - accuracy: 0.8232
Epoch 5/25
800/800 [==============================] - 1s 1ms/step - loss: 0.4020 - accuracy: 0.8216
Epoch 6/25
800/800 [==============================] - 1s 1ms/step - loss: 0.3893 - accuracy: 0.8321
Epoch 7/25
800/800 [==============================] - 1s 999us/step - loss: 0.3716 - accuracy: 0.8487
Epoch 8/25
800/800 [==============================] - 1s 1ms/step - loss: 0.3797 - accuracy: 0.8446
Epoch 9/25
800/800 [==============================] - 1s 1ms/step - loss: 0.3659 - accuracy: 0.8521
Epoch 10/25
800/800 [==============================] - 1s 1ms/step - loss: 0.3661 - accuracy: 0.85

In [ ]:
# Predicting a single new observation
"""
Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000
"""
new_prediction = classifier.predict(sc.transform(np.array([[600, 0, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)



#Importing few libraries for k-folds
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense


def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 1)

accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv =10, n_jobs = -1 )
mean1 = accuracies.mean()
variance = accuracies.std()

In [ ]:
#Improvising the ANN - Hyper parameter optimization

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10)) 
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier


classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [10,15],
              'epochs': [1,5]}

grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', n_jobs = -1, cv = 10)
grid_search = grid_search.fit(X_train, y_train)
grid_search.best_params_
grid_search.best_score_

Epoch 1/5
800/800 [==============================] - 1s 974us/step - loss: 0.5667 - accuracy: 0.8010
Epoch 2/5
800/800 [==============================] - 1s 1ms/step - loss: 0.4358 - accuracy: 0.7956
Epoch 3/5
800/800 [==============================] - 1s 1ms/step - loss: 0.4345 - accuracy: 0.7968
Epoch 4/5
800/800 [==============================] - 1s 1ms/step - loss: 0.4170 - accuracy: 0.8287
Epoch 5/5
800/800 [==============================] - 1s 1ms/step - loss: 0.4162 - accuracy: 0.8313


0.827875

## Keras-tuner

We are going to begin by setting up the virtual environment for `keras-tuner` by installing the library.


In [ ]:
!pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.2)


Now that we have that downloaded, let's import all the necessary libraries.

In [ ]:
import kerastuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense


LOG_DIR = f"{int(time.time())}"

Creating a hypermodel with `keras-tuner`

I'm going to rename the classifier in the code above to model.

In [ ]:
def build_model(hp):
    model = keras.models.Sequential()
    model.add(Dense(units = hp.Int('inputUnits', min_value=6, max_value=12), kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    
    for i in range(hp.Int("numLayers", min_value=1, max_value=4)):
      model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    model.compile(
      optimizer = tf.keras.optimizers.Adam(hp.Choice('learnRate', [0.01,0.001,0.02,0.001])),
      loss = 'binary_crossentropy',
      metrics = ['acc']
    )

    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials = 5,
    executions_per_trial = 3,
    directory = LOG_DIR
)

In [ ]:
tuner.search(
    x = X_train,
    y = y_train,
    epochs = 10,
    validation_data = (X_test, y_test)
)

Trial 5 Complete [00h 00m 15s]
val_acc: 0.8575000166893005

Best val_acc So Far: 0.8575000166893005
Total elapsed time: 00h 01m 16s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in 1610802763/untitled_project
Showing 10 best trials
Objective(name='val_acc', direction='max')
Trial summary
Hyperparameters:
inputUnits: 11
numLayers: 2
learnRate: 0.02
Score: 0.8580000003178915
Trial summary
Hyperparameters:
inputUnits: 9
numLayers: 4
learnRate: 0.01
Score: 0.8501666784286499
Trial summary
Hyperparameters:
inputUnits: 10
numLayers: 1
learnRate: 0.01
Score: 0.8423333366711935
Trial summary
Hyperparameters:
inputUnits: 9
numLayers: 4
learnRate: 0.001
Score: 0.8399999936421713
Trial summary
Hyperparameters:
inputUnits: 12
numLayers: 1
learnRate: 0.001
Score: 0.8399999936421713


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'inputUnits': 11, 'learnRate': 0.02, 'numLayers': 2}

In [ ]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11)                121       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 242
Trainable params: 242
Non-trainable params: 0
_________________________________________________________________


Saving the best model we have got so far 

In [ ]:
import pickle

with open(f'tuner_{int(time.time())}.pkl', 'wb') as f:
    pickle.dump(tuner, f)

In [ ]:
!ls

1610751939  1610802763	ANN-with-keras-tuner.ipynb  tuner_1610802841.pkl
1610751997  ann.py	Churn_Modelling.csv


In [ ]:
tmp_tuner = pickle.load(open('tuner_1610802841.pkl','rb'))

In [ ]:
tmp_tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11)                121       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 242
Trainable params: 242
Non-trainable params: 0
_________________________________________________________________


### Conclusion

That's the end of this tutorial implmenting ANN from scratch and modifying your code by using keras-tuner.
If you liked it and learned something new, let us know. And if there are any more topics you'd like us to cover, do reach out to us regarding that as well.

Have an amazing time ahead.